In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
full_data = pd.read_csv("../../data/processed/historical/train/lcl_data.csv")
df_25K = full_data.sample(25000, random_state=0)
df_25K.to_csv("../../data/processed/historical/train/lcl_data_25K.csv", index=False)

# Load Data

In [3]:
import torch
import numpy as np
import random
RANDOM_STATE = 0
torch.manual_seed(RANDOM_STATE)
torch.use_deterministic_algorithms(True)
g = torch.Generator()
g.manual_seed(RANDOM_STATE)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [4]:
from pathlib import Path
from opensynth.data_modules.lcl_data_module import LCLDataModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt

data_path = Path("../../data/processed/historical/train/lcl_data_25K.csv")
stats_path = Path("../../data/processed/historical/train/mean_std.csv")
outlier_path = Path("../../data/processed/historical/train/outliers.csv")

dm = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=25000, n_samples=25000)
dm.setup()

In [5]:
import torch
from opensynth.models.faraday import FaradayVAE
vae_model = torch.load("vae_model.pt")
vae_model.eval()

/var/folders/w2/2x1gj41j1xb5z153q5m_4rn80000gn/T/ipykernel_90772/2311655816.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae_model = torch.load("vae_model.pt")


FaradayVAE(
  (encoder): Encoder(
    (encoder_layers): Sequential(
      (0): Linear(in_features=50, out_features=512, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): GELU(approximate='none')
      (4): Linear(in_features=256, out_features=128, bias=True)
      (5): GELU(approximate='none')
      (6): Linear(in_features=128, out_features=64, bias=True)
      (7): GELU(approximate='none')
      (8): Linear(in_features=64, out_features=32, bias=True)
      (9): GELU(approximate='none')
      (10): Linear(in_features=32, out_features=16, bias=True)
    )
  )
  (decoder): Decoder(
    (latent): Linear(in_features=18, out_features=16, bias=True)
    (latent_activations): GELU(approximate='none')
    (decoder_layers): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): GELU(approximate='no

In [6]:
from opensynth.models.faraday.gaussian_mixture.prepare_gmm_input import encode_data_for_gmm

next_batch = next(iter(dm.train_dataloader()))
input_tensor = encode_data_for_gmm(data=next_batch, vae_module=vae_model)
input_data = input_tensor.detach().numpy()
n_samples = len(input_tensor)

In [7]:
N_COMPONENTS = 250
REG_COVAR = 1e-4
EPOCHS = 25
IDX = 0
CONVERGENCE_TOL = 1e-2


In [8]:
input_tensor[0][0]

tensor(0.0195, grad_fn=<SelectBackward0>)

# Init GMM

In [9]:
from opensynth.models.faraday.new_gmm import gmm_utils

labels_, means_, responsibilities_ = gmm_utils.initialise_centroids(
        X=input_data, n_components=N_COMPONENTS
    )
print(labels_.dtype, responsibilities_.dtype, means_.dtype)

torch.float32 torch.float32 torch.float32


In [10]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params

gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
print(gmm_init_params["precision_cholesky"][IDX][0][0])
print(gmm_init_params["weights"].sum())

tensor(4.1566)
tensor(1.)


# Torch GMM

In [11]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params, training_loop
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureModel


gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
torch_gmm = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR,
    print_idx=IDX
)
torch_gmm.initialise(gmm_init_params)
trained_model = training_loop(model=torch_gmm, data=input_tensor, max_iter=EPOCHS)

 72%|███████▏  | 18/25 [00:05<00:02,  3.31it/s]

Converged: True. Number of iterations: 18


# SK Learn GMM Manual

In [12]:
import numpy as np
from scipy.special import logsumexp
from scipy import linalg

def _estimate_gaussian_parameters(X, resp, reg_covar=REG_COVAR):
    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]
    n_components, n_features = means.shape
    covariances = np.empty((n_components, n_features, n_features))
    for k in range(n_components):
        diff = X - means[k]
        covariances[k] = np.dot(resp[:, k] * diff.T, diff) / nk[k]
        covariances[k].flat[:: n_features + 1] += reg_covar
    return nk, means, covariances

def _compute_precision_cholesky(covariances):
    estimate_precision_error_message = (
        "Fitting the mixture model failed because some components have "
        "ill-defined empirical covariance (for instance caused by singleton "
        "or collapsed samples). Try to decrease the number of components, "
        "or increase reg_covar."
    )

    n_components, n_features, _ = covariances.shape
    precisions_chol = np.empty((n_components, n_features, n_features))
    for k, covariance in enumerate(covariances):
        try:
            cov_chol = linalg.cholesky(covariance, lower=True)
        except linalg.LinAlgError:
            raise ValueError(estimate_precision_error_message)
        precisions_chol[k] = linalg.solve_triangular(
            cov_chol, np.eye(n_features), lower=True
        ).T
    return precisions_chol

def _compute_log_det_cholesky(matrix_chol, n_features):
    n_components, _, _ = matrix_chol.shape
    log_det_chol = np.sum(
        np.log(matrix_chol.reshape(n_components, -1)[:, :: n_features + 1]), 1
    )
    return log_det_chol

def _estimate_log_gaussian_prob(X, means, precisions_chol):
    n_samples, n_features = X.shape
    n_components, _ = means.shape

    log_det = _compute_log_det_cholesky(precisions_chol, n_features)

    log_prob = np.empty((n_samples, n_components))
    for k, (mu, prec_chol) in enumerate(zip(means, precisions_chol)):
        y = np.dot(X, prec_chol) - np.dot(mu, prec_chol)
        log_prob[:, k] = np.sum(np.square(y), axis=1)
    return -0.5 * (n_features * np.log(2 * np.pi) + log_prob) + log_det

def _estimate_log_weights(weights):
        return np.log(weights)

def _estimate_weighted_log_prob(X, means, precisions_chol, weights):
        return _estimate_log_gaussian_prob(X, means, precisions_chol) + _estimate_log_weights(weights)


def _estimate_log_prob_resp(X, means, precisions_chol, weights):
    weighted_log_prob = _estimate_weighted_log_prob(X, means, precisions_chol, weights)
    log_prob_norm = logsumexp(weighted_log_prob, axis=1)
    with np.errstate(under="ignore"):
        log_resp = weighted_log_prob - log_prob_norm[:, np.newaxis]
    return log_prob_norm, log_resp

def _e_step(X,means, precisions_chol, weights):
    log_prob_norm, log_resp = _estimate_log_prob_resp(X, means, precisions_chol, weights)
    return np.mean(log_prob_norm), log_resp

def _m_step(X, log_reponsibilities, reg_covar=REG_COVAR):

    weights_, means_, covariances_ = _estimate_gaussian_parameters(X,np.exp(log_reponsibilities),reg_covar=reg_covar)
    weights_ /= weights_.sum()

    precision_cholesky_ = _compute_precision_cholesky(covariances=covariances_)

    return precision_cholesky_, weights_, means_, covariances_

In [72]:
means = gmm_init_params["means"].detach().numpy()
weights = gmm_init_params["weights"].detach().numpy()
prec_chol = gmm_init_params["precision_cholesky"].detach().numpy()

print(f"Initial prec chol: {prec_chol[IDX][0][0]}. Initial mean: {means[IDX][0]}")

converged = False
lower_bound = -np.inf

for i in range(EPOCHS):
    prev_lower_bound = lower_bound

    print(f"Old Prec Chol: {prec_chol[IDX][0][0]}. Old means: {means[IDX][0]}")
    log_prob, log_resp = _e_step(input_data, means, prec_chol, weights)
    prec_chol, weights, means, covar = _m_step(input_data, log_resp)

    print(f"New prec chol: {prec_chol[IDX][0][0]}. New means: {means[IDX][0]}")

    # Converegence
    lower_bound = log_prob
    change = abs(lower_bound - prev_lower_bound)
    print(f"Change: {change}")
    if change < CONVERGENCE_TOL:
        converged = True
        break

print(f'Converged: {converged}. Number of iterations: {i}')

Initial prec chol: 4.1548590660095215. Initial mean: 0.17847225069999695
Old Prec Chol: 4.1548590660095215. Old means: 0.17847225069999695
New prec chol: 4.257102611679153. New means: 0.15923378387399825
Change: inf
Old Prec Chol: 4.257102611679153. Old means: 0.15923378387399825
New prec chol: 4.327276987105484. New means: 0.134237492363237
Change: 10.72411567783233
Old Prec Chol: 4.327276987105484. Old means: 0.134237492363237
New prec chol: 4.164612669379692. New means: 0.11181758419310123
Change: 0.2380362878273523
Old Prec Chol: 4.164612669379692. Old means: 0.11181758419310123
New prec chol: 3.9273389292878718. New means: 0.09253723395509501
Change: 0.15503535082142328
Old Prec Chol: 3.9273389292878718. Old means: 0.09253723395509501
New prec chol: 3.805008424813823. New means: 0.08124611012317282
Change: 0.11686195655645015
Old Prec Chol: 3.805008424813823. Old means: 0.08124611012317282
New prec chol: 3.760100553900476. New means: 0.07088212095724039
Change: 0.09799915202646581

# SK Learn GMM Epoch

In [14]:
from sklearn.mixture import GaussianMixture

init_weights = gmm_init_params["weights"]
init_means = gmm_init_params["means"]

skgmm = GaussianMixture(n_components=N_COMPONENTS, covariance_type='full', tol=CONVERGENCE_TOL, max_iter=EPOCHS, random_state=0, means_init = init_means, weights_init=init_weights)
skgmm.fit(input_data)
skgmm_pred = skgmm.predict(input_data)

# Torch Lightning

In [13]:
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import LightningDataModule
class CustomDataset(Dataset):
    def __init__(self, data_tensor: torch.Tensor):
        self.data = data_tensor
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    
class CustomDataModule(LightningDataModule):
    def __init__(self, data_tensor: torch.Tensor, batch_size: int):
        super().__init__()
        self.data_tensor = data_tensor
        self.batch_size = batch_size
    def setup(self, stage=""):
        self.custom_ds = CustomDataset(self.data_tensor)
    def train_dataloader(self):
        return DataLoader(self.custom_ds, batch_size=self.batch_size, shuffle=False, generator=g, worker_init_fn=seed_worker)
    
custom_dm = CustomDataModule(data_tensor=input_tensor, batch_size=25000)
custom_dm.setup(stage="")

In [14]:

for i in range(5):
    print(next(iter(custom_dm.train_dataloader()))[0][0])

tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)


In [15]:
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureLightningModule
gmm_module = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR,
    print_idx=IDX
)
gmm_module.initialise(gmm_init_params)
gmm_lightning_module = GaussianMixtureLightningModule(
    gmm_module = gmm_module,
    vae_module = vae_model,
    num_components = gmm_module.num_components,
    num_features = gmm_module.num_features,
    reg_covar = gmm_module.reg_covar,
    convergence_tolerance = CONVERGENCE_TOL
)
trainer = pl.Trainer(max_epochs=EPOCHS, accelerator="cpu", deterministic=True )
trainer.fit(gmm_lightning_module, custom_dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/core/opt

Epoch 18: 100%|██████████| 1/1 [00:00<00:00,  2.55it/s, v_num=9]


# Compare

In [16]:
IDX = 0

In [123]:
skgmm.means_[IDX]

array([-0.01986567, -1.58313106, -0.8376821 ,  1.50835015,  0.53980565,
       -0.07691685, -0.5145912 ,  3.14849379, -0.2685238 , -1.63142382,
       -0.45010982, -1.37293559, -2.26417557,  0.96674663,  1.11909963,
       -1.18587405,  7.20388948,  2.8516499 ])

In [17]:
trained_model.means[IDX]

tensor([-0.0291, -1.5898, -0.8670,  1.5469,  0.5635, -0.0816, -0.5269,  3.3206,
        -0.2614, -1.6546, -0.4404, -1.3841, -2.4280,  1.0509,  1.1529, -1.1667,
         7.2701,  2.8376])

In [125]:
means[IDX]

array([ 0.01319139, -1.84227717, -1.0187363 ,  1.75468597,  0.51564125,
       -0.08966222, -0.75001982,  3.537216  , -0.29240853, -1.94970427,
       -0.59990003, -1.46578683, -2.34941918,  1.05371177,  1.13516882,
       -1.37207263,  7.6638798 ,  2.85942797])

In [18]:
gmm_lightning_module.gmm_module.means[IDX]

tensor([-0.0291, -1.5898, -0.8670,  1.5469,  0.5635, -0.0816, -0.5269,  3.3206,
        -0.2614, -1.6546, -0.4404, -1.3841, -2.4280,  1.0509,  1.1529, -1.1667,
         7.2701,  2.8376])

In [127]:
gmm_init_params["means"][IDX]

tensor([ 0.1785, -2.7549, -1.0892,  1.7319,  0.2749, -0.0532, -0.9788,  3.2775,
        -0.2790, -2.2910, -0.6581, -1.5413, -1.2498,  0.9345,  0.9561, -2.0828,
         5.9111,  3.9389])

In [128]:
skgmm.precisions_cholesky_[IDX][0]

array([ 3.55173626,  2.53793189,  2.96998557, -1.00798125, -3.35486382,
       -5.59920729,  2.77419047, 11.58124173,  1.20359304, -4.91899616,
        2.70120213,  3.327976  , -6.34447119, -5.22434905,  2.24990343,
        3.8607954 ,  4.43594485,  5.0263799 ])

In [129]:
trained_model.precision_cholesky[IDX][0]

tensor([ 4.1157,  3.0687,  2.2776, -1.0621, -3.1639, -4.0429,  4.4531, 12.6161,
         0.8838, -6.8453,  3.7621,  5.3410, -7.1727, -5.7260,  6.5221,  5.1429,
         0.8379,  4.9380])

In [130]:
prec_chol[IDX][0]

array([ 3.7897146 ,  2.46545725,  2.78797016, -1.36522727, -2.8992264 ,
       -4.10797488,  3.75299131, 12.48509889,  0.6537569 , -5.57916278,
        2.87689676,  4.99061726, -5.54004165, -4.69811518,  5.59457284,
        1.02480001,  0.27719458,  7.89269578])

In [131]:
gmm_lightning_module.gmm_module.precision_cholesky[IDX][0]

tensor([ 4.1157,  3.0687,  2.2776, -1.0621, -3.1639, -4.0429,  4.4531, 12.6161,
         0.8838, -6.8453,  3.7621,  5.3410, -7.1727, -5.7260,  6.5221,  5.1429,
         0.8379,  4.9380])

In [132]:
gmm_init_params["precision_cholesky"][IDX][0]

tensor([ 4.1549,  1.6064,  2.5591, -2.7706, -3.7916, -4.3451,  2.0203,  6.5932,
        -0.2606, -2.5117, -0.2632,  3.9858, -2.3138, -2.7559, -1.5492, -0.6929,
         2.3419, -0.1891])

# Sampling

In [86]:
def sample(means_, covariances_, weights_, n_samples):
    rng = np.random.RandomState(RANDOM_STATE)
    n_samples_comp = rng.multinomial(n_samples, weights_)

    X = np.vstack(
            [
                rng.multivariate_normal(mean, covariance, int(sample))
                for (mean, covariance, sample) in zip(
                    means_, covariances_, n_samples_comp
                )
            ]
        )
    
    y = np.concatenate(
        [np.full(sample, j, dtype=int) for j, sample in enumerate(n_samples_comp)]
    )
    return (X, y)

In [87]:
N_SAMPLES = 250

In [113]:
samples = sample(means, covar, weights, n_samples = N_SAMPLES)

X, y = samples
X[IDX], y[IDX]

(array([-0.11805573, -0.89393653, -0.77229941,  1.99134082,  1.89012784,
         0.0666209 , -0.43303208,  3.07549095, -0.21847191, -2.88415176,
        -0.8482324 , -1.32996758, -3.09175821,  0.67537419,  2.47094925,
        -1.50222956,  5.09321157,  4.78674957]),
 0)

In [115]:
train_model_samples = sample(trained_model.means.detach().numpy(), trained_model.covariances.detach().numpy(), trained_model.weights.detach().numpy(), n_samples = N_SAMPLES)
train_model_X, train_model_y = train_model_samples
train_model_X[IDX], train_model_y[IDX]

/var/folders/w2/2x1gj41j1xb5z153q5m_4rn80000gn/T/ipykernel_83245/1110536824.py:7: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rng.multivariate_normal(mean, covariance, int(sample))


(array([-0.17359757, -1.55022903, -0.62314378,  1.74089907,  1.85032463,
         0.01132923, -0.11662867,  3.85972572, -0.01647614, -3.72196875,
        -0.30405943, -0.32018143, -3.18765051,  1.13523552,  3.03087308,
        -2.01908667,  5.15243561,  2.45702508]),
 0)

In [120]:
gmm_lightning_samples = sample(gmm_lightning_module.gmm_module.means.detach().numpy(), gmm_lightning_module.gmm_module.covariances.detach().numpy(), gmm_lightning_module.gmm_module.weights.detach().numpy(), n_samples = N_SAMPLES)
gmm_lightning_X, gmm_lightning_y = train_model_samples
gmm_lightning_X[IDX], gmm_lightning_y[IDX]

/var/folders/w2/2x1gj41j1xb5z153q5m_4rn80000gn/T/ipykernel_83245/1110536824.py:7: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rng.multivariate_normal(mean, covariance, int(sample))


(array([-0.17359757, -1.55022903, -0.62314378,  1.74089907,  1.85032463,
         0.01132923, -0.11662867,  3.85972572, -0.01647614, -3.72196875,
        -0.30405943, -0.32018143, -3.18765051,  1.13523552,  3.03087308,
        -2.01908667,  5.15243561,  2.45702508]),
 0)